In [5]:
import pandas as pd

# tweaking settings to see all columns on screen
pd.set_option('display.max_columns', None)

import numpy as np
import time
import math
from datetime import timedelta 
from datetime import datetime

N = 3 # number of lanes
FT_MI = 5280 # mile to feet conversion
TAO = 1.75 #
SPEED_CONVER = 5280/3600 # ft/s to mph conversion
L = 14.75 # typical length of a car
l = 5 # loop length
DEPARTURE_CONVER = 120 # hourly departure of v vehicles; raw data shows vehicles per 30 seconds

In [16]:
DATE

datetime.datetime(2021, 2, 11, 0, 0)

# Data Cleaning

In [6]:
df = pd.read_csv('78 EB.csv')

# Record the Date when the data was recorded
DATE = df.loc[0, 'Date']
DATE = datetime.strptime(DATE, '%Y/%m/%d')

df.drop(columns=['Route', 'Direction', 'pcw1occ', 'Mainline Lanes', 'Date'], inplace=True)

# rename columns
df.rename(columns={"pcw1speed": "V_ups",
                   "pcw1vol": "Q_ups",
                   "MlDetPostmile": "postmile"}, inplace = True)

# select ['Name']s with word 'to'
df = df[df.Name.str.contains('to')] # those that dont have 'to' is radar data; no ramp data
df['Name'] = df['Name'].apply(lambda x: x.split('to')[0][:-1])

# convert datatype of 'time' to datetime 
def convtime(s):
    return datetime.strptime(s, '%H:%M:%S').time()

df['Time'] = df['Time'].apply(convtime)

# Derive q_exit
df['q_exit'] = (df['r_1_offcnt'] + df['r_2_offcnt']) * DEPARTURE_CONVER
df.drop(['r_1_offcnt', 'r_2_offcnt'], axis=1, inplace = True)

# Derive q_ramp
df['q_ramp'] = (df['Psg 1 Cnt'] + df['Psg 2 Cnt'] + df['Psg 3 Cnt']) * DEPARTURE_CONVER
df.drop(['Psg 1 Cnt', 'Psg 2 Cnt', 'Psg 3 Cnt'], axis=1, inplace = True)

# Derive V_merge
V_ups = df['V_ups']
Q_ups = df['Q_ups']
q_ramp = df['q_ramp']

df['V_merge'] = (FT_MI*N*V_ups)/(TAO*SPEED_CONVER*((N*Q_ups)+q_ramp))-(L/(TAO*SPEED_CONVER))
df['V_merge'] = df[['V_ups','V_merge']].min(axis=1)

# convert Nan values to 0
df['q_exit'] = df['q_exit'].fillna(0)
df['q_ramp'] = df['q_ramp'].fillna(0)
df['V_ups'] = df['V_ups'].fillna(0)
df['Q_ups'] = df['Q_ups'].fillna(0)

# change all V_ups value of 0 to the previous data
df['V_ups'] = df['V_ups'].replace(to_replace=0, method='ffill')

# Rearrange table
df = df[['Id', 'Name', 'Time', 'V_ups', 'Q_ups', 'q_ramp', 'q_exit', 'V_merge', 'postmile']]
df.reset_index(drop=True, inplace=True)
df.sort_values(by=['Time', 'postmile'])

# Rename table
raw = df.copy()
df = None

# V_ups: upstream speed in mph
# Q_ups: upstream volume in lane/hr
# q_ramp: discharge rate; rate at which cars enter the highway per hour at a certain ramp
# q_exit: rate at which cars exit a certain ramp
# postmile: number of miles a ramp is into the highway

In [7]:
# Shift up q_exit values and calculate V_ds and Q_ds
# To do a calculation for ramp1 at time T, we need q_exit value of ramp2 at T+(1min)
# V_ds and Q_ds of ramp 1 at time T are V_ups and Q_ups of ramp 2 at time T+(1min)

df = raw.copy()

times = df['Time'].unique().tolist()
ramps = df['Id'].unique().tolist()

temp_df = pd.DataFrame()

for ramp in ramps:
    ramp_df = df.loc[df['Id'] == ramp]
    
    q_exits = ramp_df['q_exit'].tolist()
    q_exits = q_exits[1:] + [None]
    ramp_df.loc[:, 'q_exit'] = q_exits
    
    Q_ds = ramp_df['Q_ups'].tolist()
    Q_ds = Q_ds[1:] + [None]
    ramp_df.loc[:, 'Q_ds'] = Q_ds
    
    V_ds = ramp_df['V_ups'].tolist()
    V_ds = V_ds[1:] + [None]
    ramp_df.loc[:, 'V_ds'] = V_ds
    
    
    
    temp_df = pd.concat([temp_df, ramp_df])
    
df = temp_df.copy()
temp_df = pd.DataFrame()

for time in times:
    time_df = df.loc[df['Time'] == time]
    
    q_exits = time_df['q_exit'].tolist()
    q_exits = q_exits[1:] + [None]
    time_df.loc[:, 'q_exit'] = q_exits
    
    Q_ds = time_df['Q_ups'].tolist()
    Q_ds = Q_ds[1:] + [None]
    time_df.loc[:, 'Q_ds'] = Q_ds
    
    V_ds = time_df['V_ups'].tolist()
    V_ds = V_ds[1:] + [None]
    time_df.loc[:, 'V_ds'] = V_ds
    
    temp_df = pd.concat([temp_df, time_df])
    
    
temp_df.sort_values(by=['Time', 'postmile'], inplace=True)
temp_df.reset_index(drop=True)

df = temp_df.copy()

df = df[df['q_exit'].notna()]
df = df[df['Q_ds'].notna()]
df = df[df['V_ds'].notna()]

# Rename table
raw = df.copy()
df = None

raw

,Id,Name,Time,V_ups,Q_ups,q_ramp,q_exit,V_merge,postmile,Q_ds,V_ds
0,201,Jefferson St,05:30:00,73,213,0.0,0.0,73.0,0.858,335.0,63.0
1,200,El Camino Real,05:30:00,63,335,0.0,0.0,63.0,1.594,274.0,59.0
2,199,Plaza Dr,05:30:00,59,274,0.0,120.0,59.0,3.586,335.0,52.0
3,24,Emerald Dr,05:30:00,52,335,0.0,120.0,52.0,4.474,517.0,79.0
4,205,Vista Village Dr,05:30:00,79,517,600.0,120.0,79.0,6.320,548.0,78.0
...,...,...,...,...,...,...,...,...,...,...,...
12594,398,Las Posas Rd/Grand Ave,19:29:00,82,975,0.0,0.0,82.0,11.364,761.0,66.0
12595,180,San Marcos Blvd,19:29:00,66,761,0.0,480.0,66.0,12.274,914.0,68.0
12596,234,Twin Oaks Vlly Rd,19:29:00,68,914,360.0,0.0,68.0,13.022,670.0,67.0
12597,236,Barham/Woodland,19:29:00,67,670,0.0,360.0,67.0,14.860,853.0,62.0


# Tensorflow

In [8]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import matplotlib.pyplot as plt
from tqdm import tqdm
from keras.preprocessing import image
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

### Build Dataset

In [6]:
# split training set
X = raw[['Q_ups', 'V_ups', 'q_ramp', 'q_exit']]
y = raw[['Q_ds', 'V_ds']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [7]:
# Fit model
def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    
    tf.keras.optimizers.Adam(
        learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
        name='Adam'
    )
    
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy']
                 )
    
    return model

model = get_compiled_model()
model.fit(X_train, y_train, epochs=15)

Epoch 1/15
294/294 [==============================] - 1s 1ms/step - loss: -2587475.0582 - accuracy: 0.9521
Epoch 2/15
294/294 [==============================] - 0s 1ms/step - loss: -84029700.7458 - accuracy: 0.9524
Epoch 3/15
294/294 [==============================] - 0s 2ms/step - loss: -499444815.6203 - accuracy: 0.9543
Epoch 4/15
294/294 [==============================] - 0s 1ms/step - loss: -1456761880.7322 - accuracy: 0.9516
Epoch 5/15
294/294 [==============================] - 0s 1ms/step - loss: -3087155725.0169 - accuracy: 0.9533
Epoch 6/15
294/294 [==============================] - 0s 2ms/step - loss: -5575070206.2644 - accuracy: 0.9566
Epoch 7/15
294/294 [==============================] - 1s 2ms/step - loss: -8870906050.3864 - accuracy: 0.9566
Epoch 8/15
294/294 [==============================] - 0s 2ms/step - loss: -13168722548.2847 - accuracy: 0.9525
Epoch 9/15
294/294 [==============================] - 0s 1ms/step - loss: -18469040367.5119 - accuracy: 0.9535
Epoch 10/15
29

In [8]:
# Evaluate Model
model.evaluate(X_test, y_test)

74/74 [==============================] - 0s 950us/step - loss: -84409925632.0000 - accuracy: 0.9541


[-84409925632.0, 0.954081654548645]

### Our model has successfully been trained.

# Calculation

### Clean Data for Particle Filter

In [20]:
# calculate n_ups
df = raw.copy()
df.drop(columns=['q_ramp'], inplace=True)

V_ups = df['V_ups']
Q_ups = df['Q_ups']
q_exit = df['q_exit']
V_merge = df['V_merge']

n_ups = Q_ups/V_ups
df['n_ups'] = n_ups

In [21]:
# calculate n_goal

# Shift up n_ups values
# To do calculations for n_goal of ramp2 at time T, we need n_ups value of ramp1 at T-(1min)

times = df['Time'].unique().tolist()
ramps = df['Id'].unique().tolist()

temp_df = pd.DataFrame()

for ramp in ramps:
    ramp_df = df.loc[df['Id'] == ramp]
    
    n_goal = ramp_df['n_ups'].tolist()
    n_goal = n_goal[:-1]
    n_goal = [None] + n_goal 
    ramp_df.loc[:, 'n_goal'] = n_goal
    
    temp_df = pd.concat([temp_df, ramp_df])
    
df = temp_df.copy()
temp_df = pd.DataFrame()

for time in times:
    time_df = df.loc[df['Time'] == time]
    
    n_goal = time_df['n_ups'].tolist()
    n_goal = [None] + n_goal[:-1]
    n_goal = n_goal
    time_df.loc[:, 'n_goal'] = n_goal
    
    temp_df = pd.concat([temp_df, time_df])
    
temp_df['n_goal'] = temp_df['n_goal'] * 1.15
    
temp_df.sort_values(by=['Time', 'postmile'], inplace=True)
temp_df.reset_index(drop=True)

df = temp_df.copy()

df = df[df['n_goal'].notna()]

### Calculate the optimal q_ramp from a range of randomly generated q_ramps

In [22]:
# the larger the weight, the more efficient the q_ramp
def weight(n_ds, Q_ups, V_ups):
    n_ups = Q_ups/V_ups
    n_goal = 1.15 * n_ups
    temp = n_ds-n_goal
    R = 10 # arbitrary variable for normal Gaussian distribution 
    return (1/((2*math.pi*R)**(1/2))) * math.e**((-1)*((n_ds-n_goal)**2)/(2*R))

In [23]:
# calculate optimizing q_ramp from random q_ramp samples

# load arbitraty q_ramp values
q_ramps = [1008,971,934,896,859,822,785,748,710,673,636,599,562,524,484]
q_ramps = [i* 2 for i in q_ramps] # two ramps => double the discharge

V_ups = df['V_ups']
Q_ups = df['Q_ups']
q_exit = df['q_exit']
V_merge = df['V_merge']
n_goal = df['n_goal']

R = 1 # arbitrary
cols = [] # list to store column names of weights

for q_ramp in q_ramps:
    n_ds = (1/V_ups)*(Q_ups+(q_ramp/N))-(1/N)*(q_exit/V_merge)
    weight_val = weight(n_ds, Q_ups, V_ups)
    col = 'weight_'+str(q_ramp)
    cols.append(col)
    df[col] = weight_val
    
# choose the largest/best weight across multiple columns
df = df.assign(q_ramp=df[cols].idxmax(axis=1))
df = df[df['q_ramp'].notna()]
df['q_ramp'] = df['q_ramp'].apply(lambda x: float(x.split('_')[1]))


df = df[['Id', 'Name', 'Q_ups', 'Time', 'V_ups', 'q_exit', 'V_merge', 'postmile', 'Q_ds', 'V_ds', 'n_ups', 'q_ramp']]

intermediate = df.copy()

### Generate a normal distribution from the optimal q_ramp calculated above to find accurate, optimal q_ramp

In [13]:
# mu: mean; best_rate
# sigma: standard deviation
def generate_range(mu, sigma=2):
    return np.random.normal(mu, sigma, 15).tolist()

In [14]:
q_ramps = df.q_ramp.unique()

new_df = pd.DataFrame()

# for each q_ramp value
for q_ramp in q_ramps[0:1]:
    temp_df = df.loc[df['q_ramp'] == q_ramp]
    # generate a new set of q_ramps using normal distribution to fine-tune the q_ramp value
    normal_distribution = generate_range(q_ramp)
    
    V_ups = temp_df['V_ups']
    Q_ups = temp_df['Q_ups']
    q_exit = temp_df['q_exit']
    V_merge = temp_df['V_merge']
    
    # calculate new weight using the new q_ramp values
    cols = []
    for new_q_ramp in normal_distribution:
        n_ds = (1/V_ups)*(Q_ups+(new_q_ramp/N))-(1/N)*(q_exit/V_merge)
        weight_val = weight(n_ds, Q_ups, V_ups)
        col = 'weight_'+str(new_q_ramp)
        cols.append(col)
        temp_df[col] = weight_val
        
    # choose the largest/best weight across multiple columns
    temp_df = temp_df.assign(q_ramp=temp_df[cols].idxmax(axis=1))
    temp_df = temp_df[temp_df['q_ramp'].notna()]
    temp_df['q_ramp'] = temp_df['q_ramp'].apply(lambda x: float(x.split('_')[1]))
    temp_df.drop(columns=cols, inplace=True)
        
    new_df = pd.concat([new_df, temp_df])
    
new_df.sort_values(by=['Time', 'postmile'])
particleFilter = new_df['q_ramp'].tolist()

/Users/taekunkim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Compare Particle Filter against TensorFlow

In [24]:
intermediate

,Id,Name,Q_ups,Time,V_ups,q_exit,V_merge,postmile,Q_ds,V_ds,n_ups,q_ramp
1,200,El Camino Real,335,05:30:00,63,0.0,63.0,1.594,274.0,59.0,5.317460,968.0
2,199,Plaza Dr,274,05:30:00,59,120.0,59.0,3.586,335.0,52.0,4.644068,968.0
3,24,Emerald Dr,335,05:30:00,52,120.0,52.0,4.474,517.0,79.0,6.442308,968.0
4,205,Vista Village Dr,517,05:30:00,79,120.0,79.0,6.320,548.0,78.0,6.544304,968.0
5,198,Escondido Ave,548,05:30:00,78,0.0,78.0,6.886,853.0,71.0,7.025641,968.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12594,398,Las Posas Rd/Grand Ave,975,19:29:00,82,0.0,82.0,11.364,761.0,66.0,11.890244,968.0
12595,180,San Marcos Blvd,761,19:29:00,66,480.0,66.0,12.274,914.0,68.0,11.530303,968.0
12596,234,Twin Oaks Vlly Rd,914,19:29:00,68,0.0,68.0,13.022,670.0,67.0,13.441176,968.0
12597,236,Barham/Woodland,670,19:29:00,67,360.0,67.0,14.860,853.0,62.0,10.000000,968.0


# Visualization

In [ ]:
# libraries
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
toPlot = raw.copy()

time = "17:30:00"



# calculate distance between each ramp
toPlot = toPlot.sort_values('postmile')
toPlot['distance'] = toPlot[['postmile']].diff(periods=1)['postmile']
df[-1, 'distance'] = 0.0

# calculate travel time
toPlot['travel_time'] = toPlot['V_ups'] * toPlot['distance']

# calculate volume
toPlot['volume'] = toPlot['V_ups'] * toPlot['n_ds']

# calculate occupancy
toPlot['occupancy'] = (toPlot['n_ds']*(L+l))/(5280+(toPlot['n_ds']+l))

# drop unnecessary columns
toPlot = toPlot.drop(columns = ['Id', 'Q_ups', 'convtime', 'Time', 'q_exit', 'q_ramp', 'n_ds', 'postmile'])

# rename columns
toPlot.rename(columns={"V_ups": "Speed(mph)", 
                       "distance": "Distance(mi)", 
                       "occupancy": "Occupancy(%)",
                       "volume": "Volume",
                       "travel_time": "Travel Time(min)"}, inplace=True)

original_toPlot = toPlot.copy()

In [ ]:
toPlot

In [ ]:
labels = [
          'Speed(mph)',
          'Occupancy(%) X 2',
          'Distance(mi) X 100',
          'Travel Time(min)',
          'Volume X 10'
          ]
toPlot['Occupancy(%)'] = toPlot['Occupancy(%)']*200
toPlot['Occupancy(%)'] = toPlot['Occupancy(%)']*200
toPlot['Distance(mi)'] = toPlot['Distance(mi)']*100
toPlot['Volume'] = toPlot['Volume']*1/10

toPlot.reset_index(inplace=True)


  
# Change the style of plot
plt.style.use('seaborn-dark')


# Set the background colors
fig = plt.figure()
fig.patch.set_facecolor('whitesmoke')

ax = fig.add_subplot(111)
ax.set_facecolor("whitesmoke")

# Initialize x_labels
x_label = toPlot.pop('Name')

# Initialize colors
colors = plotcolor = ['hotpink','orange','darkgoldenrod','lightseagreen','orchid']
 
# Add titles
plt.title("Highway CA-78", loc='left', fontsize=15, fontweight='bold', color='black')
plt.xlabel("Ramp, in the order of traffic flow", fontsize=10, fontweight='bold')
plt.xticks(rotation = 45)
# # Show the graph

num=0
for column, name, color in zip(toPlot.columns[1:], labels, colors): # start from 1 to avoid index from being plotted
    num+=1
    plt.plot(x_label, toPlot[column], marker='', color=color, linewidth=1.2, alpha=0.9, label=name)
    
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(9)

plt.axis([0, len(x_label), 0, 300])
plt.legend(loc=2, ncol=2, prop={'size': 9})

annotation = "data recorded on " + date + " " + time[:-3]
plt.figtext(0, -0.03, annotation, fontsize=7)


plt.tight_layout()
fig1 = plt.gcf()
plt.show()
plt.draw()
plt.gcf().subplots_adjust(bottom=0.45)
fig1.savefig('graph.png', dpi=500)

toPlot = original_toPlot

In [ ]:
# plot speed

# Change the style of plot
plt.style.use('seaborn-dark')

# Set the background colors
fig = plt.figure()
fig.patch.set_facecolor('whitesmoke')

ax = fig.add_subplot(111)
ax.set_facecolor("whitesmoke")

# Initialize colors
colors = plotcolor = ['hotpink','orange','darkgoldenrod','lightseagreen','orchid']

# Initialize x_labels
# defined in previous block

# Add titles
plt.title("Highway CA-78 Speed", loc='left', fontsize=15, fontweight='bold', color='black')
plt.xlabel("Ramp, in the order of traffic flow", fontsize=10, fontweight='bold')
plt.xticks(rotation = 45)

# plot graph
plt.plot(x_label, toPlot['Speed(mph)'], marker='', color='orange', linewidth=1.2, alpha=0.9, label='Speed(mph)')
    
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(9)

plt.axis([0, len(x_label), 0, 130])
plt.legend(loc=2, ncol=2, prop={'size': 9})

annotation = "data recorded on " + date + " " + time[:-3]
plt.figtext(0, -0.03, annotation, fontsize=7)


plt.tight_layout()
fig1 = plt.gcf()
plt.show()
plt.draw()
plt.gcf().subplots_adjust(bottom=0.45)
fig1.savefig('speed.png', dpi=500)

toPlot = original_toPlot

In [ ]:
# plot distance

# Change the style of plot
plt.style.use('seaborn-dark')

# Set the background colors
fig = plt.figure()
fig.patch.set_facecolor('whitesmoke')

ax = fig.add_subplot(111)
ax.set_facecolor("whitesmoke")

# Initialize colors
colors = plotcolor = ['hotpink','orange','darkgoldenrod','lightseagreen','orchid']

# Initialize x_labels
# x_label = toPlot.pop('Name')

# Add titles
plt.title("Highway CA-78 Distance", loc='left', fontsize=15, fontweight='bold', color='black')
plt.xlabel("Ramp, in the order of traffic flow", fontsize=10, fontweight='bold')
plt.xticks(rotation = 45)

# plot graph
plt.plot(x_label, toPlot['Distance(mi)'], marker='', color='orange', linewidth=1.2, alpha=0.9, label='Distance(mi)')
    
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(9)

plt.axis([0, len(x_label), 0, 3])
plt.legend(loc=2, ncol=2, prop={'size': 9})

annotation = "data recorded on " + date + " " + time[:-3]
plt.figtext(0, -0.03, annotation, fontsize=7)


plt.tight_layout()
fig1 = plt.gcf()
plt.show()
plt.draw()
plt.gcf().subplots_adjust(bottom=0.45)
fig1.savefig('distance.png', dpi=500)

toPlot = original_toPlot

In [ ]:
# plot distance

# Change the style of plot
plt.style.use('seaborn-dark')

# Set the background colors
fig = plt.figure()
fig.patch.set_facecolor('whitesmoke')

ax = fig.add_subplot(111)
ax.set_facecolor("whitesmoke")

# Initialize colors
colors = plotcolor = ['hotpink','orange','darkgoldenrod','lightseagreen','orchid']

# Initialize x_labels
# x_label = toPlot.pop('Name')

# Add titles
plt.title("Highway CA-78 Occupancy", loc='left', fontsize=15, fontweight='bold', color='black')
plt.xlabel("Ramp, in the order of traffic flow", fontsize=10, fontweight='bold')
plt.xticks(rotation = 45)

# plot graph
plt.plot(x_label, toPlot['Occupancy(%)'], marker='', color='orange', linewidth=1.2, alpha=0.9, label='Occupancy(%)')
    
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(9)

plt.axis([0, len(x_label), 0, 0.5])
plt.legend(loc=2, ncol=2, prop={'size': 9})

annotation = "data recorded on " + date + " " + time[:-3]
plt.figtext(0, -0.03, annotation, fontsize=7)


plt.tight_layout()
fig1 = plt.gcf()
plt.show()
plt.draw()
plt.gcf().subplots_adjust(bottom=0.45)
fig1.savefig('occupancy.png', dpi=500)

toPlot = original_toPlot

In [ ]:
# plot distance

# Change the style of plot
plt.style.use('seaborn-dark')

# Set the background colors
fig = plt.figure()
fig.patch.set_facecolor('whitesmoke')

ax = fig.add_subplot(111)
ax.set_facecolor("whitesmoke")

# Initialize colors
colors = plotcolor = ['hotpink','orange','darkgoldenrod','lightseagreen','orchid']

# Initialize x_labels
# x_label = toPlot.pop('Name')

# Add titles
plt.title("Highway CA-78 Volume", loc='left', fontsize=15, fontweight='bold', color='black')
plt.xlabel("Ramp, in the order of traffic flow", fontsize=10, fontweight='bold')
plt.xticks(rotation = 45)

# plot graph
plt.plot(x_label, toPlot['Volume'], marker='', color='orange', linewidth=1.2, alpha=0.9, label='Volume')
    
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(9)

plt.axis([0, len(x_label), 0, 3000])
plt.legend(loc=2, ncol=2, prop={'size': 9})

annotation = "data recorded on " + date + " " + time[:-3]
plt.figtext(0, -0.03, annotation, fontsize=7)


plt.tight_layout()
fig1 = plt.gcf()
plt.show()
plt.draw()
plt.gcf().subplots_adjust(bottom=0.45)
fig1.savefig('volume.png', dpi=500)

toPlot = original_toPlot

In [ ]:
# plot distance

# Change the style of plot
plt.style.use('seaborn-dark')

# Set the background colors
fig = plt.figure()
fig.patch.set_facecolor('whitesmoke')

ax = fig.add_subplot(111)
ax.set_facecolor("whitesmoke")

# Initialize colors
colors = plotcolor = ['hotpink','orange','darkgoldenrod','lightseagreen','orchid']

# Initialize x_labels
# x_label = toPlot.pop('Name')

# Add titles
plt.title("Highway CA-78 Travel Time", loc='left', fontsize=15, fontweight='bold', color='black')
plt.xlabel("Ramp, in the order of traffic flow", fontsize=10, fontweight='bold')
plt.xticks(rotation = 45)

# plot graph
plt.plot(x_label, toPlot['Travel Time(min)'], marker='', color='orange', linewidth=1.2, alpha=0.9, label='Travel Time(min)')
    
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(9)

plt.axis([0, len(x_label), 0, 200])
plt.legend(loc=2, ncol=2, prop={'size': 9})

annotation = "data recorded on " + date + " " + time[:-3]
plt.figtext(0, -0.03, annotation, fontsize=7)


plt.tight_layout()
fig1 = plt.gcf()
plt.show()
plt.draw()
plt.gcf().subplots_adjust(bottom=0.45)
fig1.savefig('travel_time.png', dpi=500)

toPlot = original_toPlot

In [ ]:
toPlot

In [ ]:
# # Dataframe
# toPlot = matrix.loc[matrix['Id'] == 201]
# toPlot = toPlot[['V_ups', 'Time']]
# toPlot['Time'] = pd.to_datetime(toPlot.Time)
# toPlot = toPlot.groupby(pd.Grouper(key='Time', freq='30min')).mean().dropna()
  
# # Change the style of plot
# plt.style.use('seaborn-darkgrid')
 
# # Create a color palette
# palette = plt.get_cmap('Set1')
 
# # Plot multiple lines
# num=0
# for column in toPlot.columns:
#     num+=1
#     plt.plot(toPlot.index.hour, toPlot[column], marker='', color=palette(num), linewidth=1, alpha=0.9, label=column)

# # Add legend
# plt.legend(loc=2, ncol=2)
 
# # Add titles
# plt.title("Speed VS Time", loc='left', fontsize=12, fontweight=0, color='orange')
# plt.xlabel("Hour")
# plt.xticks(rotation = 45)
# plt.ylabel("Speed (mph)")
# plt.ylim(0, 80)

# # Show the graph
# plt.show()

# UNEXPECTED VALUE RETURNED FROM MODEL:

In [ ]:
print('Expected:')
print(y_test)
print('What the model returns:')
print(model.predict(X_test))

In [ ]:
simple_test = df.iloc[1, :9].to_frame().transpose().reset_index()
simple_test

In [ ]:
test_df = pd.DataFrame()

q_ramps = [1008,971,934,896,859,822,785,748,710,673,636,599,562,524,484]   # hypothetical q_ramp
Q_ups = simple_test.loc[0, 'Q_ups']
V_ups = simple_test.loc[0, 'V_ups']
q_exit = simple_test.loc[0, 'q_exit']

test_df['q_ramp'] = q_ramps
test_df['Q_ups'] = Q_ups
test_df['V_ups'] = V_ups
test_df['q_exit'] = q_exit

test_df

### Our model accurately calculates Q_ds and V_ds. 
### We are now ready to compare our particle filter model with the DL model.

In [ ]:
model(test_df)